<div align="center">

# **Influence Cascade Decoding (ICD)**  

### *Notebook finale: IR-NLP & Analisi di Social Network e Media*  

---

**Ivan Prisco**  
**Vincenzo Presta**  
**Matteo Greco**  

---
</div>


# 1. Setup
Queste celle si occupano di tutta la configurazione iniziale necessaria.
 
- Installare le librerie principali (`bitsandbytes`, `transformers`, `accelerate`).
- Mostrare le informazioni sulla GPU disponibile con `nvidia-smi`.
- Configurare la memoria CUDA per evitare errori di allocazione.
- Recuperare in modo sicuro il **token Hugging Face** dai *Kaggle Secrets*.
- Eseguire il login su Hugging Face e stampa le info dell’account.
- Definire una funzione `load_model()` che:
  - carica il **tokenizer**,
  - carica il modello (in questo caso **Mistral-7B-Instruct**) in quantizzazione 4-bit (per risparmiare memoria GPU),
  - abilita gli **hidden states** per eventuali analisi avanzate.

In [ ]:
!pip install -qU transformers accelerate bitsandbytes 
!nvidia-smi 

In [ ]:
import torch
import networkx as nx
import matplotlib.pyplot as plt
import os
import math
import pickle
from kaggle_secrets import UserSecretsClient   
from huggingface_hub import login, whoami      
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, util
import numpy as np
import re
import unicodedata
import random
from collections import defaultdict
import nltk
from nltk.corpus import stopwords 
import json
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import time
from typing import Dict
from transformers import LogitsProcessor, LogitsProcessorList
from pathlib import Path

In [ ]:
# config memoria GPU
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

#--- Hugginface
user_secrets = UserSecretsClient()  
hf_token = user_secrets.get_secret("HF_TOKEN")
os.environ["HF_TOKEN"] = hf_token
login(token=hf_token)

info = whoami()
print(f"Login effettuato come: {info['name']}")

In [ ]:
#--- Modello
model_id = "mistralai/Mistral-7B-Instruct-v0.3" 

def load_model(model_id):
    bnb_config = BitsAndBytesConfig(load_in_4bit=True)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",               
        quantization_config=bnb_config,  
        dtype="auto",                    
        attn_implementation="sdpa"      
    )
    return tokenizer, model
    
tokenizer, model = load_model(model_id)
model.config.output_hidden_states = True  

---

## Baseline Decoding
### Funzione di baseline per la generazione

Si definisce una funzione che permette di generare testo in modo standard utilizzando le tecniche di decoding già integrate nel modello.  
Questa baseline sarà utile come punto di riferimento per confrontare le prestazioni e le differenze rispetto a nuove strategie di decoding che verranno introdotte in seguito.  


In [ ]:
def baseline_decoding(model, tokenizer, input_text, max_new_tokens=200, temperature=0.7, top_k=50):
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

    # Usa il token di padding corretto, o l'eos_token se non presente 
    pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_k=top_k,
            num_return_sequences=1,
            pad_token_id=pad_token_id,
            no_repeat_ngram_size=2
        )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return output_text

---

# 2. Creazione della rete

## 2.1 Strategie di simulazione dei token

Questa sezione implementa diverse **strategie di campionamento** dei token a partire
da una distribuzione di probabilità generata dal modello. L’obiettivo è simulare
diversi scenari di generazione, così da analizzare in seguito le transizioni tra token
e costruire le reti di influenza.

### 1. Nucleus Sampling (*Top-p*)
- Ordina i token in base alla probabilità.
- Calcola la somma cumulata delle probabilità fino a superare la soglia `p` (tipicamente 0.9).
- Considera solo quell’insieme ristretto di token (“nucleus”) e ne sceglie uno in modo casuale.
  
### 2. Temperature Sampling
- Applica un fattore di **temperatura** alla distribuzione:
  - `T < 1`: distribuzione più “spinta” verso i token più probabili → testi più conservativi.
  - `T > 1`: distribuzione più piatta → maggiore esplorazione e creatività.
  - `T = 0`: degenerazione in greedy decoding (si sceglie sempre il token più probabile).
- Utile per controllare direttamente il livello di variabilità nelle simulazioni.

### 3. Top-k Sampling
- Seleziona i `k` token con probabilità più alta.
- Campiona casualmente un token solo da questo sottoinsieme.
---

### Funzione `run_all_simulations`
Questa funzione orchestra l’intero processo di simulazione:

- Riceve in input un **prompt iniziale** e parametri di configurazione (`num_simulazioni`, `max_steps`).  
- Per ciascuna simulazione:
  1. Inizializza la sequenza con il token finale del prompt.
  2. Sceglie in modo casuale una **strategia di campionamento** tra *top-k*, *top-p* e *temperature* (con temperature diverse prese a caso).
  3. Itera per un numero massimo di passi (`max_steps`), generando un token per volta.  
  4. Salva per ogni passo una tupla `(token_id, probabilità)`.  

- Restituisce una **matrice di simulazioni**, organizzata come una lista di liste:  
  - Ogni **riga** rappresenta una simulazione completa (dall’inizio alla fine).  
  - All’interno di ogni riga, i **passi della simulazione** sono salvati in ordine sequenziale.  
  - Ogni passo è rappresentato da una **tupla** `(token_id, probabilità)`:
    - `token_id`: identificativo numerico del token scelto dal vocabolario del modello,
    - `probabilità`: valore della distribuzione al momento dell’estrazione di quel token.  

---

Qui viene posta la base sperimentale per trasformare la generazione del modello in un problema di **analisi di rete**, utile per studiare il ruolo dei token e la propagazione dell’influenza.


In [ ]:
prompt = "The future of AI is"

In [ ]:
#i metodi prendono la distribuzione dei token e poi successivamente scelgono la strategia

def nucleus_sampling(probs, top_p=0.9): #1 STRATEGIA DI SIMULAZIONE
    sorted_indices = np.argsort(probs)[::-1]
    sorted_probs = probs[sorted_indices]
    cumulative_probs = np.cumsum(sorted_probs)
    cutoff_index = np.argmax(cumulative_probs > top_p) + 1
    #trova il primo indice dove la somma cumulata supera p
    top_p_indices = sorted_indices[:cutoff_index] #prendismo solo gli indici che ricadono nell'insieme
    top_p_probs = sorted_probs[:cutoff_index]
    top_p_probs = top_p_probs / top_p_probs.sum()
 
    return np.random.choice(top_p_indices, p=top_p_probs)# estraiamo da quell'inisieme di indici un token casualmente
 
 
def temperature_sampling(probs, temperature=0.7): #2 STRATEGIA DI SIMULAZIONE
    if temperature == 0: #se temperatura=0--> greedy decoding
        return np.argmax(probs)
    probs = np.log(probs + 1e-12) / temperature #le proababilità variano in base alla temperatura, T<1 "raffredda" , T>1 appiattisce aumentando la diversità
    probs = np.exp(probs)#riportiamo in probabilita1 normali dopo che erano log probability
    probs = probs / probs.sum() #normalizziamo
    return np.random.choice(len(probs), p=probs)
 
        
def run_all_simulations(model, tokenizer, num_simulazioni, max_steps, prompt):
    all_tokens = []   # matrice simulazioni x step con (token, prob), ogni riga ha le info di ciascuna simulazione , in cui si memorizzano token e prob di token estratto
    context_tokens = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    device = context_tokens.device
 
     #  prendi l'ultimo token del prompt(lo mettiamo perchè altrimenti nella pesatura degli archi non riusciremmo ad ottenere quello che vogliamo)
    start_token = context_tokens[0, -1].item()
 
    for j in range(num_simulazioni):
        generated = context_tokens.clone()
        tokens_sim = []   # (token_id, prob)
 
        
        #  inserisci subito il token finale del prompt con prob=0.0
        tokens_sim.append((start_token, 0.0))
        # scegli una strategia fissa per la simulazione
        strategy = random.choice(["top_k", "top_p", "temp"])
        if strategy == "temp":
            temp = random.choice([0, 0.7, 1.0, 1.2, 1.5])  # esempio
        print(f"Simulazione {j} - Strategia: {strategy}")
 
        for _ in range(max_steps):
            with torch.no_grad():
                outputs = model(generated)
                logits = outputs.logits[:, -1, :]
                probs = torch.softmax(logits, dim=-1)[0]
                probs_np = probs.detach().cpu().to(torch.float32).numpy() #sposta da gpu a cpu per essere analizzato
 
 
            if strategy == "top_k":
                top_k = 50
                top_k_values, top_k_indices = torch.topk(probs, top_k)
                top_k_probs = top_k_values / top_k_values.sum()
                chosen_index = torch.multinomial(top_k_probs, 1) #campiona dai top k un token casualmente
                next_token_id = top_k_indices[chosen_index].item() #prende l'id del token estratto
            elif strategy == "top_p":
                next_token_id = nucleus_sampling(probs_np, top_p=0.9)
            else:  # temp
                next_token_id = temperature_sampling(probs_np, temperature=temp)
 
            tokens_sim.append((next_token_id, probs[next_token_id].item()))#qui generiamo il vettore in cui ciascun elemento è una tupla: (token scelto,probabilità)
 
            next_token = torch.tensor([[next_token_id]], device=device)
            generated = torch.cat((generated, next_token), dim=-1)
        #print("tokens_sim: ",tokens_sim)
 
        all_tokens.append(tokens_sim)

 
        print(" RISULTATO SIMULAZIONE", j)
        print("Token-level:", tokens_sim)
        #print("Word-level:", word_probs)
 
    return all_tokens

La cella successiva si occupa del salvataggio della matrice *all_tokens* derivante dalle simulazioni. 

In [ ]:
# parametri 
domain = "AI" #questo parametro serve a scopo esplicativo, da cambiare rispetto al dominio
num_simulazioni = 150 
max_steps = 200

# Lancio simulazioni
all_tokens= run_all_simulations(
    model, tokenizer,
    num_simulazioni=num_simulazioni,
    max_steps=max_steps,
    prompt=prompt
)

file_name = f"all_tokens_{num_simulazioni}sim_{max_steps}steps_{domain}.pkl"

with open(file_name, "wb") as f:
    pickle.dump(all_tokens, f)

print(f"Simulazioni completate e matrice esportata in: {file_name}")

---
### Caricamento della matrice
Questa cella serve a ricaricare da file la matrice delle simulazioni già esportata, evitando di dover ripetere il processo di generazione (costoso in termini di tempo e risorse).  
In questo modo si può subito disporre delle sequenze simulate e procedere ai passi successivi.


In [ ]:
path = "/kaggle/input/all-tokens-simulations/all_tokens_150sim_200steps_AI.pkl"

# Caricamento matrice con transizioni
with open(path, "rb") as f:
    all_tokens = pickle.load(f)

print("Matrice ricaricata, numero simulazioni:", len(all_tokens))
print("Esempio prima simulazione:", all_tokens[0][:5])  # primi 5 token della prima simulazion

---

## 2.2 Costruzione della rete 



### Grafo dei token

In questa cella viene definita la funzione `build_token_graph`, che trasforma le sequenze di token generate nelle simulazioni in un **grafo orientato**:

- I **nodi** rappresentano i token.  
- Gli **archi** rappresentano le transizioni osservate tra token consecutivi.  
- Il **nodo iniziale** è l’ultimo token del prompt, da cui partono le simulazioni.  

Il grafo viene quindi creato a partire da tutte le simulazioni:
- Si scorre la matrice *all_tokens*:
    - Per ogni tupla (*token_id*, *prob*) si crea un nodo *token_id* nella rete.
    - Per ogni coppia di tuple (*token_id_1*, *prob_1*), (*token_id_2*, *prob_2*) in una riga, si crea un arco orientato da *token_id_1* a *token_id_2* nella rete. 


In [ ]:
def build_token_graph(all_tokens, prompt_tokens):
    
    G = nx.DiGraph()

    # nodo iniziale = ultimo token del prompt
    start_token = prompt_tokens[0, -1].item()

    G.add_node(start_token)

    for sim in all_tokens:  # ogni simulazione
        if not sim:  # simulazione vuota → salta
            continue

        # ora scorre i token consecutivi nella simulazione
        for i in range(len(sim) - 1):
            t1, _ = sim[i]
            t2, _ = sim[i + 1]

            if t1 not in G:
                G.add_node(t1)
            if t2 not in G:
                G.add_node(t2)

            if not G.has_edge(t1, t2):
                G.add_edge(t1, t2)

    return G
    
prompt_tokens = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
G = build_token_graph(all_tokens, prompt_tokens)
print("Rete creata.")
print("Numero nodi:", G.number_of_nodes())
print("Numero archi:", G.number_of_edges())

#debug
for node in G.nodes():
    vicini = list(G.successors(node))
    #print(f"{node} → {vicini}")

In questa fase, il grafo costruito è puramente strutturale: ogni arco è presente
o assente in base alle transizioni osservate, senza alcuna informazione
quantitativa. Nella fase successiva verranno introdotti i pesi
associati agli archi, al fine di riflettere la forza o la rilevanza delle transizioni. 

### Assegnamento dei pesi agli archi della rete
Per rendere il grafo informativo e adatto a modellare processi di
diffusione, è stato introdotto uno **schema di pesatura** che combina due
componenti distinte: la stima empirica delle transizioni e la correlazione
statistica tra token.

1. **Stima condizionata $\hat{p}(v|u)$**  
   La prima componente è la **probabilità condizionata stimata** di osservare un token $v$ successivamente a un token $u$, definita come:

   $$
   \hat{p}(v|u) = \frac{\text{freq}(u \to v)}{\sum_{v'} \text{freq}(u \to v')}
   $$

   dove $\text{freq}(u \to v)$ indica il numero di volte in cui la transizione $(u,v)$ è stata osservata nelle simulazioni. Questa misura cattura l’aspetto puramente empirico della frequenza delle transizioni.

2. **PPMI (Positive Pointwise Mutual Information)**  
   La seconda componente è la \textbf{Positive Pointwise Mutual Information} (PPMI), che valuta la forza statistica dell’associazione tra due token al di là della loro frequenza marginale. È definita come:

   $$
   \text{PPMI}(u, v) = \max \left( \log \frac{p(u, v)}{p(u)\,p(v)}, \; 0 \right)
   $$

dove: 
- $p(u,v)$ è la probabilità congiunta della coppia $(u,v)$;
- $p(u)$ e $p(v)$ sono le probabilità marginali dei due token.

Per rendere i punteggi confrontabili con le probabilità condizionate, i valori di PPMI vengono normalizzati nell’intervallo $[0,1]$, dividendo per il massimo valore di PMI..

---

#### Formula finale del peso
Il peso assegnato a ciascun arco è una combinazione convessa delle due componenti:

$$
w(u, v) = \alpha \cdot \hat{p}(v|u) + (1 - \alpha) \cdot \text{PPMI}_{\text{norm}}(u, v)
$$

dove:  
- $\alpha \in [0,1]$ regola il bilanciamento tra frequenza empirica e correlazione semantica.   

Il valore finale del peso viene forzato nell’intervallo $[0, 1]$.

---

Questo approccio presenta diversi vantaggi:

- riduce il numero di archi con peso nullo, grazie alla combinazione di due misure diverse; 
- tiene conto sia della **frequenza empirica** delle transizioni osservate, sia della **forza statistica** della loro associazione;  
- fornisce una base più solida per modellare i processi di diffusione dell’influenza nella rete.


In [ ]:
from collections import defaultdict

def compute_edge_weights(G, all_tokens, alpha=0.6):

    # Dizionari per contare frequenze
    freq_uv   = defaultdict(int)    # frequenza delle transizioni u->v
    out_total = defaultdict(int)    # numero totale di uscite da u
    in_total  = defaultdict(int)    # numero totale di ingressi in v

    # Si contano tutte le transizioni nei dati
    for sim in all_tokens:  # sim = [(token_id, prob), ...]
        for i in range(len(sim) - 1):
            u = sim[i][0]
            v = sim[i + 1][0]
            freq_uv[(u, v)] += 1
            out_total[u]   += 1
            in_total[v]    += 1

    total_bigrams = sum(freq_uv.values()) or 1 # Totale dei bigrammi osservati

    # Probabilità marginali per u e v
    p_u = {u: out_total[u] / total_bigrams for u in out_total}
    p_v = {v: in_total[v]  / total_bigrams for v in in_total}

    # Calcola PPMI per ogni arco osservato
    eps = 1e-12
    pmi_vals = {}
    for (u, v), c in freq_uv.items():
        p_uv = c / total_bigrams
        pmi  = math.log((p_uv + eps) / ((p_u.get(u, 0) + eps) * (p_v.get(v, 0) + eps)))
        pmi_vals[(u, v)] = max(pmi, 0.0)  # PPMI (solo valori positivi)

    # Normalizza PPMI in [0,1]
    max_ppmi = max(pmi_vals.values()) if pmi_vals else 1.0

    def scale_ppmi(x):
        return 0.0 if max_ppmi <= 0 else (x / max_ppmi)

    # Funzione per la stima condizionata p_hat(v|u)
    def p_cond(u, v):
        denom = out_total.get(u, 0)
        return freq_uv.get((u, v), 0) / denom if denom > 0 else 0.0

    # Assegna a ogni arco i pesi
    for u, v in G.edges():
        cond = p_cond(u, v)
        pp = scale_ppmi(pmi_vals.get((u, v), 0.0))
        w  = alpha * cond + (1 - alpha) * pp
        G[u][v]['p_cond']  = cond
        G[u][v]['ppmi']   = pp
        G[u][v]['weight'] = min(1.0, max(0.0, w))  # forzo tra 0 e 1

    return G


In questo contesto, si è scelto di fissare il valore $\alpha = 0.4$ per la costruzione definitiva della rete. Tale valore rappresenta un compromesso equilibrato: da un lato evita una rete eccessivamente polarizzata, dominata da archi molto forti accanto a numerosi archi debolissimi; dall’altro non conduce a una struttura troppo sparsa o vuota.
La distribuzione risultante appare dunque sufficientemente centrale, mantenendo un buon numero di archi attivi e fornendo una base solida per la fase di diffusione.

In [ ]:
G = compute_edge_weights(G, all_tokens, alpha=0.4)

count = 0
# Visualizzazione dei pesi
for u, v, data in G.edges(data=True):
    print(f"{u} → {v} : p_cond={data['p_cond']:.3f}, ppmi={data['ppmi']:.3f}, weight={data['weight']:.3f}")
    count += 1 
    if count >= 10: 
        break

### PLOT: Distribuzione dei pesi

In [ ]:
weights = [data["weight"] for _, _, data in G.edges(data=True)]

# plot istogramma
plt.figure(figsize=(6,4))
plt.hist(weights, bins=30, color="skyblue", edgecolor="black")
plt.xlabel("Peso arco")
plt.ylabel("Frequenza")
plt.title("Distribuzione dei pesi degli archi")
plt.show()

Come precedentemente accennato, per $\alpha = 0.4$ la distribuzione si allarga e mostra una maggiore variabilità. L’effetto combinato di frequenza e PPMI genera una gamma di pesi più diversificata, con un picco intorno a 0.2

---

# 3. Diffusione nella rete

Dopo aver costruito il grafo e assegnato i pesi agli archi, la fase successiva consiste nell’analisi della propagazione dell’influenza tra i token. Per questo scopo è stato adottato il modello di diffusione **Independent Cascade (IC)**, particolarmente adatto a contesti in cui le attivazioni avvengono in modo probabilistico lungo gli archi pesati della rete.

I **seed**, ovvero i nodi che risultano essere attivi all'inizio del processo di diffusione sono stati definiti come i token contenuti nel prompt di partenza *{"The future of AI is"}*. Essi rappresentano le sorgenti di influenza da cui prende avvio la diffusione nella rete.

##  3.1 Estrazione dei seed

In questa cella viene definita la funzione `get_prompt_seeds`, che ha lo scopo di estrarre i **nodi seed** a partire dal prompt:

- Il prompt viene **tokenizzato** e ciascun token viene associato al relativo ID.  
- Ogni token del prompt viene aggiunto al grafo `G` come nodo (se non già presente).  
- Viene creata la lista `seed_nodes`, che rappresenta i nodi di partenza per la diffusione dell’influenza.  
- Per chiarezza, vengono stampati a schermo gli ID dei token seed insieme alla loro decodifica testuale.  

In questo modo la diffusione nel grafo inizia direttamente dai token presenti nel prompt fornito.


In [ ]:
def get_prompt_seeds(G, tokenizer, prompt):
    # tokenizza il prompt
    prompt_tokens = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)["input_ids"][0].tolist()
    
    seed_nodes = []
    for tid in prompt_tokens:
        if tid not in G:
            G.add_node(tid)  # aggiungi nodo mancante
        seed_nodes.append(tid)

    # stampa i seed trovati
    print("Token seed dal prompt")
    for tid in seed_nodes:
        print(f"{tid}:'{tokenizer.decode([tid])}'")


    return seed_nodes, G

## 3.2 Independent Cascade

Il modello IC si basa su una dinamica stocastica di attivazione dei nodi e ha le seguenti caratteristiche:

- **Seed iniziali**: si parte da un insieme di nodi già attivi (nel nostro caso, i token del prompt).  
- **Propagazione**: ad ogni step temporale, ogni nodo che è diventato attivo al passo precedente prova ad attivare i suoi vicini.  
- **Probabilità di attivazione**: il tentativo di attivazione su un arco `(u → v)` ha successo con probabilità pari al peso dell’arco. In questo caso, i pesi derivano dalle simulazioni di generazione (frequenza × probabilità media della transizione).  
- **Una sola occasione**: ogni nodo attivo ha una sola opportunità di attivare ciascun vicino. Se fallisce, non potrà più ritentare in futuro.  
- **Terminazione**: il processo continua finché non ci sono più nuovi nodi attivati.  

Per stimare in modo stabile l’influenza di ciascun nodo, il processo viene eseguito **molte volte (Monte Carlo)** e si calcola, per ogni nodo, la frazione di simulazioni in cui risulta attivato.  

Infatti, per ciascun nodo $v$ viene calcolata la *probabilità empirica di attivazione*, definita come:
$$
p_{\text{att}}(v) =
\frac{\# \,\mathrm{volte\ in\ cui}\ v\ \mathrm{è\ attivo}}
{\mathrm{numero\ totale\ di\ simulazioni}}
$$
In questo modo si ottiene un punteggio di influenza globale `influence_score[node] ∈ [0,1]` che misura quanto il nodo è “raggiungibile” dai seed secondo la dinamica IC.

In [ ]:
def independent_cascade(G, seeds, num_iter=1000):
    """
    Simula Independent Cascade su grafo pesato.
    Restituisce un dizionario {nodo: punteggio_influenza}
    """
    influence_count = defaultdict(int)

    for _ in range(num_iter):
        active = set(seeds)        # nodi attivi all'inizio
        newly_active = set(seeds)

        while newly_active:
            next_new = set()
            for u in newly_active:
                for v in G.successors(u):
                    if v not in active:
                        prob = G[u][v].get("weight", 0.0)
                        if random.random() < prob:  # attivazione con prob
                            next_new.add(v)
            newly_active = next_new
            active.update(newly_active)

        # aggiorna conteggio influenza
        for node in active:
            influence_count[node] += 1

    # normalizza in [0,1]
    influence_score = {node: influence_count[node] / num_iter for node in G.nodes()} #è un dizionario
    return influence_score

# seeds dal prompt
seed_nodes, G = get_prompt_seeds(G, tokenizer, prompt)

# simulazione IC
scores = independent_cascade(G, seed_nodes, num_iter=1000)

# classifica i nodi più influenzati
ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
print("\n--- Nodi più influenzati ---")
for node, score in ranked[:100]:
    print(f"{node}:'{tokenizer.decode([node])}' → {score:.3f}")

## 3.3  Costruzione del logit bias dai punteggi IC

Costruita la rete di influenza e definiti i pesi degli archi, il passo successivo consiste nell’integrare queste informazioni all’interno del processo di generazione del modello linguistico. L’idea è di condizionare i \textit{logit}
prodotti dal modello in fase di decoding, in modo che la distribuzione finale dei token candidati rifletta non solo la conoscenza statistica del modello ma anche la struttura del grafo e eventuali bias esterni.


---

In questa cella vengono definiti gli strumenti per convertire i **punteggi di influenza (IC scores)** in un **bias** da applicare al modello:

- Vengono scaricate le **stopword inglesi** tramite `nltk` e definite le principali **punteggiature**.  
- La funzione `fixScore`:
  - riceve in input un dizionario di punteggi IC per token,  
  - decodifica ciascun token per identificarne il contenuto testuale,  
  - assegna un **bias pieno** ai token considerati concetti importanti,  
  - assegna un **bias ridotto (×0.8)** a stopword e punteggiatura,  
  - calcola infine un dizionario `{token_id: bias}`.

In questo modo si modulano le probabilità di generazione, riducendo l’impatto di stopword e segni di punteggiatura a favore di token più informativi. Si noti che, anche se non si fosse applicato questo leggero fix agli score, il risultato sarebbe stato molto simile in quanto la generazione non si basa esclusivamente sulla rete ma anche sui punteggi interni del modello.

In [ ]:
nltk.download("stopwords")

stop_words = set(stopwords.words("english")) 
# set di punteggiatura comuni
punctuation_tokens = {

    ",", ".", ";", ":", "-", "–", "—", "(", ")", "[", "]", "{", "}",

    "!", "?", "¿", "¡", "'", '"', "…", "...", "``", "''",

    "/", "\\", "|", "_", "+", "*", "&", "%", "$", "#", "@", "^",

    "<", ">", "=", "~", "`","’"}
 
def fixScore(scores, tokenizer):
    """
    Trasforma i punteggi IC (scores) in un dizionario {token_id: bias}
    con pesatura differenziata:
      - concetti: bias pieno
      - stopword/punteggiatura: bias ridotto
    """
    score_bias = {}
    for token_id, score in scores.items():
        token_txt = tokenizer.decode([token_id]).lower().strip()
        if not token_txt:  # salta token vuoti
            continue
        # stopword o punteggiatura → bias ridotto
        if token_txt in stop_words or token_txt in punctuation_tokens:
            factor = 0.8
        else:
            factor = 1.0
        # calcolo bias
        bias = score * factor
        score_bias[token_id] = bias
    return score_bias

In [ ]:
score_bias = fixScore(scores, tokenizer)

# ordina per bias decrescente
ranked_bias = sorted(score_bias.items(), key=lambda x: x[1], reverse=True)

print("\n--- Top 50 logit bias ---")
for tid, bias in ranked_bias[:50]:
    txt = tokenizer.decode([tid])
    print(f"{tid}:'{txt}' → {bias:.3f}")

---

# 4. Generazione

Il processo di generazione può essere descritto nei seguenti passi:

- **Codifica del prompt**: Il prompt viene tokenizzato e fornito al modello per avviare la generazione.

- **Calcolo dei logit**: a ogni passo di decoding, il modello produce un vettore di logit, cioè valori non normalizzati che rappresentano la propensione del modello a generare ciascun token del vocabolario.

- **Selezione dei candidati**: Per rendere il calcolo più efficiente e mirato, vengono considerati i **100 token con logit più alto** secondo il modello.

- **Applicazione del boosting** 
    Per ciascun candidato $v$ vengono calcolati due contributi:
    - Peso dal grafo} $w(u,v)$: se esiste un arco dal token corrente $u$ verso $v$, si utilizza il peso calcolato inf fase di costruzione della rete.
    - Bias esterno $\text{bias}(v)$: punteggio assegnato al token $v$ dal processo di diffusione, che misura la probabilità empirica di attivazione del token a partire dai seed del prompt. Questo valore permette di enfatizzare i token che risultano più centrali nella propagazione dell’influenza nella rete.
        
Questi due contributi vengono combinati e scalati dal parametro $\lambda$, ottenendo così la **formula finale**:
$$
    \text{logit\_final}(v) = \text{logit}(v) + \lambda \cdot (w(u,v) + \text{bias}(v)).
$$

- **Campionamento**: dopo la modifica dei logit, si applica una *softmax* per ottenere la nuova distribuzione di probabilità e si campiona il prossimo token tra i **50 candidati più probabili** (strategia top k).

- **Iterazioni**: il processo si ripete per il numero di passi prestabilito, generando così una sequenza in cui ogni scelta è influenzata sia dal modello che dalla rete di influenza.

In [ ]:
def generate_with_logit_boosting(model, tokenizer, prompt, G, score_bias, lambda_=2.0, steps=200):
    generated = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
 
    for step in range(steps):
        with torch.no_grad():
            outputs = model(generated)
            logits = outputs.logits[:, -1, :].squeeze()
 
        # ultimo token della sequenza
        current_token = generated[0, -1].item()
 
        # prendi i top-100 candidati
        top100 = torch.topk(logits, 100)
        indices = top100.indices.tolist()
        values = top100.values.tolist()
 
        # stampa i top-20 PRIMA
        print(f"\n=== Step {step+1} ===")
        print("--- Top 20 PRIMA boosting ---")
        for idx, val in zip(top100.indices[:20].tolist(), top100.values[:20].tolist()):
            print(f"{idx}:'{tokenizer.decode([idx])}' → logit={val:.3f}")
        
        # applica boosting solo sui top-100
        boosted_logits = logits.clone()
        for idx, val in zip(indices, values):
            infl = score_bias.get(idx, 0.0)
            w = 0.0
            if current_token in G and idx in G[current_token]:
                w = G[current_token][idx].get("weight", 0.0)
            boost = w + infl
            boosted_logits[idx] = val + lambda_ * boost
        
        # softmax per probabilità DOPO boosting
        boosted_probs = torch.softmax(boosted_logits, dim=-1)
        
        # stampa i top-20 DOPO
        top20_after = torch.topk(boosted_logits, 20)
        print("\n--- Top 20 DOPO boosting ---")
        for idx, val in zip(top20_after.indices.tolist(), top20_after.values.tolist()):
            print(f"{idx}:'{tokenizer.decode([idx])}' → logit={val:.3f}")

        # prendi i top-50 dopo boosting
        top50 = torch.topk(boosted_logits, 50)
        final_indices = top50.indices
        final_values = top50.values
 
        # campiona il prossimo token dai top-50
        probs = torch.softmax(final_values, dim=-1)
        chosen_index = torch.multinomial(probs, 1).item()
        next_token_id = final_indices[chosen_index].item()
 
        # aggiungi il token generato
        next_token = torch.tensor([[next_token_id]], device=generated.device)
        generated = torch.cat((generated, next_token), dim=-1)
 
        # stampa step
        print(f"\nStep {step+1}")
        print("Scelto:", next_token_id, "→", tokenizer.decode([next_token_id]))
 
    return tokenizer.decode(generated[0], skip_special_tokens=True)

In [ ]:
text = generate_with_logit_boosting(model, tokenizer, "The future of AI is", G, score_bias, lambda_= 1.5, steps=200)
print("\n--- Testo finale ---")
print(text)

# 5. Analisi e test dei gradi di libertà

## 5.1 Analisi e valutazione del parametro $\alpha$
La funzione `analyze_and_save_directed_basic` calcola e salva in un file JSON una serie di metriche strutturali e dinamiche di un grafo orientato pesato `G`. L’analisi viene condotta **in funzione del parametro $\alpha$** utilizzato nello
**schema di pesatura degli archi**, così da confrontare come cambia la struttura della rete e il processo di diffusione.


####  Metriche calcolate

1. Statistiche base
    - **`num_nodes`**: numero di nodi $|V|$.  
    - **`num_edges`**: numero di archi $|E|$.  
    - **`density_directed`**: densità del grafo:
  $$
  \text{density} = \frac{|E|}{|V| \cdot (|V|-1)}
  $$

2. Strength dei nodi
Somma pesata degli archi entranti/uscenti:
- $s^{in}(u) = \sum_{(v,u)\in E} w(v,u)$  
- $s^{out}(u) = \sum_{(u,v)\in E} w(u,v)$  

Si calcolano:
- **`avg_in_strength`** = media di $s^{in}(u)$  
- **`avg_out_strength`** = media di $s^{out}(u)$  

3. Statistiche sui pesi
Analisi della distribuzione dei pesi $w(u,v)$:
- $\overline{w}$ (media), Var$(w)$ (varianza), max/min, mediana, quartili $Q_1,Q_3$  

4. Connettività
    - **`num_weakly_connected_components`**: numero di componenti se si ignora l’orientamento.  
    - **`num_strongly_connected_components`**: componenti fortemente connesse.  

5. Diffusione con Independent Cascade
Si esegue il modello **Independent Cascade (IC)** con $200$ run, partendo dai token del prompt come seed.  
Per ciascun $\alpha$ si calcolano:
    - **Coverage**:
      $$
      \text{Cov} = \frac{1}{|V|} \sum_{v\in V} p_{\text{att}}(v)
      $$
      cioè la frazione media di nodi attivati.  

    - **Entropy**:
      $$
       \text{H} = - \sum_{v \in V} p_{\text{att}}(v) \log p_{\text{att}}(v)
      $$
      misura della diversità: alta se l’attivazione è diffusa, bassa se è concentrata.
  
  
---
Ripetendo queste analisi per più valori di $\alpha$, si osserva come il bilanciamento tra **frequenza empirica** e **associazione semantica** modifica:
- la struttura statica della rete (densità, pesi, strength, connettività),  
- la dinamica di diffusione (coverage, entropia, nodi centrali).  

Questo permette di identificare i valori di $\alpha$ che generano reti equilibrate ed efficaci per il boosting durante la generazione testuale.


In [ ]:
def analyze_and_save_directed_basic(G, tokenizer, alpha=None, filename="network_metrics.json", top_k=10):
    metrics = {}
    metrics["alpha"] = alpha

    # statistiche base
    metrics["num_nodes"] = int(G.number_of_nodes())
    metrics["num_edges"] = int(G.number_of_edges())
    metrics["density_directed"] = float(nx.density(G))

    # strength (somma pesi entranti e uscenti)
    in_strength = dict(G.in_degree(weight="weight"))
    out_strength = dict(G.out_degree(weight="weight"))

    metrics["avg_in_strength"] = float(np.mean(list(in_strength.values())))
    metrics["avg_out_strength"] = float(np.mean(list(out_strength.values())))


    # statistiche sui pesi
    weights = [d["weight"] for _, _, d in G.edges(data=True) if "weight" in d]
    if weights:
        metrics["avg_weight"] = float(np.mean(weights))
        metrics["var_weight"] = float(np.var(weights))
        metrics["max_weight"] = float(np.max(weights))
        metrics["min_weight"] = float(np.min(weights))
        metrics["median_weight"] = float(np.median(weights))
        metrics["q25_weight"] = float(np.quantile(weights, 0.25))
        metrics["q75_weight"] = float(np.quantile(weights, 0.75))


    # connettività
    metrics["num_weakly_connected_components"] = nx.number_weakly_connected_components(G)
    metrics["num_strongly_connected_components"] = nx.number_strongly_connected_components(G)

    # lancia simulazioni 
    scores = independent_cascade(G, seeds, num_iter=200)

    # copertura media (frazione nodi attivati)
    coverage = sum(scores.values()) / len(scores)
    results["coverage"] = coverage

    # entropia distribuzione influenza
    probs = [s for s in scores.values() if s > 0]
    entropy = -sum(p * math.log(p) for p in probs) if probs else 0.0
    results["entropy"] = entropy

    # salvataggio
    with open(filename, "w") as f:
        json.dump(metrics, f, indent=2)

    print(f" Metriche di base orientate e pesate salvate in {filename}")



### Risultati
| α    | $\overline{s^{in}}$ | $\overline{s^{out}}$ | $\overline{w}$ | Var$(w)$ | $w_{\max}$ | $w_{\min}$ | $w_{\text{med}}$ | $[Q_1,Q_3]$       |
|------|----------------------|----------------------|----------------|----------|------------|------------|------------------|-------------------|
| 0    | 1.555                | 1.555                | 0.459          | 0.070    | 1.0        | 0.0000     | 0.399            | [0.257, 0.664]    |
| 0.25 | 1.415                | 1.415                | 0.418          | 0.067    | 1.0        | 0.00018    | 0.356            | [0.221, 0.583]    |
| 0.5  | 1.276                | 1.276                | 0.377          | 0.077    | 1.0        | 0.00035    | 0.286            | [0.164, 0.546]    |
| 0.75 | 1.136                | 1.136                | 0.335          | 0.098    | 1.0        | 0.00053    | 0.202            | [0.094, 0.476]    |
| 1    | 0.996                | 0.996                | 0.294          | 0.131    | 1.0        | 0.00071    | 0.111            | [0.014, 0.500]    |


---

## 5.2 Analisi e valutazione del parametro $\lambda$ 

In questo esperimento si analizza l’effetto del parametro **λ** sul modello influenzato dalla rete.
L’obiettivo è studiare come λ modifichi la qualità e la stabilità dei testi generati, riducendo la variabilità tramite **più generazioni per ogni valore di λ**.


#### Procedura
1. **Prompt fisso**: tutte le generazioni partono dallo stesso prompt (`"The future of AI is"`).  
2. **Valori di λ**: si testano diversi valori di intensità del boosting:  
   $$\lambda \in \{0.0, 1.0, 1.5, 2.0, 2.5, 5.0, 10.0, 100.0\}$$  

   - λ **basso (≈0–0.5)** → il modello resta simile al comportamento nativo.  
   - λ **medio (1.5 - 2.5)** → la rete di influenza condiziona in modo marcato le scelte dei token.  
   - λ **alto (>5)** → rischio di collasso: la generazione segue quasi esclusivamente la rete.  
3. **Generazioni multiple**: per ogni λ si eseguono *num_runs* generazioni indipendenti (es. 30).  
   I risultati di ogni run vengono salvati in file separati (`lambda_<valore>_results.json`) contenenti testo e metriche.  

#### Metriche calcolate per ogni generazione
- **Token influenti**:  
  - conteggio assoluto $n_{\text{influenti}}$ dei token presenti nella top-50 di `ranked_bias`,  
  - frequenza normalizzata $f = \tfrac{n_{\text{influenti}}}{n_{\text{token}}}$.  

- **Perplexity (PPL)**: misura la plausibilità di una sequenza secondo una distribuzione di probabilità.  
  Data una sequenza $x = (x_1, \dots, x_N)$, la formula è:

  $$
  \text{PPL}(x) = \exp\!\Bigg(- \frac{1}{N} \sum_{t=1}^{N} \log P(x_t \mid x_{<t})\Bigg)
  $$

  dove $P(x_t \mid x_{<t})$ è la probabilità del token $x_t$ dato il contesto precedente.  

  - **Caso λ = 0** → la sequenza è generata dal modello base; la perplexity si ottiene direttamente dalla loss nativa del modello (`compute_perplexity_native`).  
  - **Caso λ > 0** → la sequenza è generata con boosting; la distribuzione viene modificata con i pesi della rete di influenza $G$ e i bias $\text{prob\_bias}$.  
    In questo caso ricostruiamo esplicitamente le probabilità boostate e calcoliamo la perplexity coerente (`compute_perplexity_boosted`).  

- **Similarità semantica**:  
  Si utilizza SBERT (`all-MiniLM-L6-v2`) per calcolare gli embedding di prompt e testo, confrontandoli tramite similarità coseno:

  $$
  \text{sim}(p, t) = \frac{ \langle e_p, e_t \rangle }{ \| e_p \| \cdot \| e_t \| }
  $$



In [ ]:
# parametri delle simulazioni
prompt = "The future of AI is"
lambda_values = [0.0, 1.0, 1.5, 2.0, 2.5, 5.0, 10.0, 100.0] 
steps = 200
num_runs = 30    # numero di generazioni per ogni λ

# modello SBERT
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

# metriche
def compute_perplexity_native(text, model, tokenizer):
    encodings = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**encodings, labels=encodings["input_ids"])
        loss = outputs.loss
    return math.exp(loss.item())

def compute_perplexity_boosted(text, model, tokenizer, G, logit_bias, lambda_=2.0):
    input_ids = tokenizer(text, return_tensors="pt")["input_ids"].to(model.device)
    n_tokens = input_ids.size(1)
    log_probs = []

    for t in range(n_tokens - 1):
        context = input_ids[:, :t+1]

        with torch.no_grad():
            outputs = model(context)
            logits = outputs.logits[:, -1, :].squeeze()

        current_token = context[0, -1].item()
        next_token = input_ids[0, t+1].item()

        boosted_logits = logits.clone()
        topk = torch.topk(logits, 100)

        for idx, val in zip(topk.indices.tolist(), topk.values.tolist()):
            infl = logit_bias.get(idx, 0.0)
            w = 0.0
            if current_token in G and idx in G[current_token]:
                w = G[current_token][idx].get("weight", 0.0)
            boost = w + infl
            boosted_logits[idx] = val + lambda_ * boost

        boosted_probs = torch.softmax(boosted_logits, dim=-1)
        prob_next = boosted_probs[next_token].item()

        log_probs.append(math.log(prob_next) if prob_next > 0 else float("-inf"))

    if len(log_probs) == 0 or float("-inf") in log_probs:
        return float("inf")

    avg_neg_log = -sum(log_probs) / len(log_probs)
    return math.exp(avg_neg_log)

influential_tokens = [tid for tid, bias in ranked_bias[:50]]

for lam in lambda_values:
    print(f"\n=== Test multipli con λ={lam} ===")
    lambda_results = []

    for run in range(num_runs):
        print(f"→ Run {run+1}/{num_runs}")

        text = generate_with_logit_boosting( 
            model, tokenizer, prompt,
            G=G, logit_bias=logit_bias,
            lambda_=lam, steps=steps  
        )

        # tokenizzazione e freq token influenti
        ids = tokenizer(text, return_tensors="pt")["input_ids"].squeeze().tolist()
        n_token = len(ids)
        n_influenti = sum(1 for t in ids if t in influential_tokens)
        freq_influenti = n_influenti / n_token if n_token > 0 else 0.0

        # perplexity
        if lam == 0.0:
            ppl = compute_perplexity_native(text, model, tokenizer)
        else:
            ppl = compute_perplexity_boosted(text, model, tokenizer, G, logit_bias, lambda_=lam)

        # similarity SBERT
        embeddings = sbert_model.encode([prompt, text], convert_to_tensor=True)
        sim = util.cos_sim(embeddings[0], embeddings[1]).item()

        # formattazione
        lambda_results.append({
            "run": run,
            "prompt": prompt,
            "lambda": lam,
            "text": text,
            "n_token": n_token,
            "n_influenti": n_influenti,
            "freq_influenti": freq_influenti,
            "perplexity": ppl,
            "sbert_similarity": sim
        })

    # salvataggio
    fname = f"lambda_{lam}_results.json"
    with open(fname, "w", encoding="utf-8") as f:
        json.dump(lambda_results, f, ensure_ascii=False, indent=2)
    print(f"Risultati salvati in {fname}")


Visualizzazione dei risultati: 

In [ ]:
# Directory del dataset
data_dir = "/kaggle/input/lambda-testing"

# Lista file
files = [
    "lambda_0.0_results.json",
    "lambda_1.0_results.json",
    "lambda_1.5_results.json",
    "lambda_2.0_results.json",
    "lambda_2.5_results.json",
    "lambda_5.0_results.json",
    "lambda_10.0_results.json",
    "lambda_100.0_results.json"
]

# === Parsing risultati ===
results = []
for file in files:
    path = os.path.join(data_dir, file)
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    freq = [d["freq_influenti"] for d in data]
    ppl = [d["perplexity"] for d in data]
    sbert = [d["sbert_similarity"] for d in data]

    results.append({
        "file": file,
        "lambda": float(file.split("_")[1].replace(".json", "")),
        "avg_freq_influenti": sum(freq) / len(freq),
        "avg_perplexity": sum(ppl) / len(ppl),
        "avg_sbert_similarity": sum(sbert) / len(sbert)
    })

df = pd.DataFrame(results).sort_values(by="lambda")
display(df)

baseline = df[df["lambda"] == 0.0].iloc[0]

# Frequenza token influenti
plt.figure(figsize=(7, 5))
plt.plot(df["lambda"], df["avg_freq_influenti"], marker="o", color="blue")
plt.axhline(y=baseline["avg_freq_influenti"], color="red", linestyle="--", label="λ=0.0 baseline")
plt.xscale("log")
plt.xticks([1, 2, 5, 10, 100], labels=[1, 2, 5, 10, 100])
plt.title("Avg Freq Influenti (log scale)", fontsize=14)
plt.xlabel("Lambda (log scale)")
plt.ylabel("Media")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.savefig("freq_influenti_log.png", dpi=300)  # <-- salva immagine
plt.show()

# Perplexity
plt.figure(figsize=(7, 5))
plt.plot(df["lambda"], df["avg_perplexity"], marker="o", color="orange")
plt.axhline(y=baseline["avg_perplexity"], color="red", linestyle="--", label="λ=0.0 baseline")
plt.xscale("log")
plt.xticks([1, 2, 5, 10, 100], labels=[1, 2, 5, 10, 100])
plt.title("Avg Perplexity (log scale)", fontsize=14)
plt.xlabel("Lambda (log scale)")
plt.ylabel("Media")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.savefig("perplexity_log.png", dpi=300)  # <-- salva immagine
plt.show()

# SBERT-similarity
plt.figure(figsize=(7, 5))
plt.plot(df["lambda"], df["avg_sbert_similarity"], marker="o", color="green")
plt.axhline(y=baseline["avg_sbert_similarity"], color="red", linestyle="--", label="λ=0.0 baseline")
plt.xscale("log")
plt.xticks([1, 2, 5, 10, 100], labels=[1, 2, 5, 10, 100])
plt.title("Avg SBERT Similarity (log scale)", fontsize=14)
plt.xlabel("Lambda (log scale)")
plt.ylabel("Media")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.savefig("sbert_similarity_log.png", dpi=300)  # <-- salva immagine
plt.show()


#### Risultati
| $\lambda$ | Freq. token influenti | Perplexity | Similarità SBERT |
|-----------|------------------------|------------|------------------|
| 0.0       | 0.0858                 | 5.760      | 0.626            |
| 1.0       | 0.1081                 | 4.467      | 0.641            |
| 1.5       | 0.1163                 | 4.230      | 0.624            |
| 2.0       | 0.1144                 | 4.369      | 0.628            |
| 2.5       | 0.1217                 | 4.083      | 0.644            |
| 5.0       | 0.1545                 | 3.087      | 0.646            |
| 10.0      | 0.2156                 | 2.608      | 0.674            |
| 100.0     | 0.3214                 | 1.230      | 0.598            |


---

# 6. Test aggiuntivi

## 6.1 A/B pair-wise test: LLM as a judge

### Introduzione

Per valutare in modo oggettivo l’efficacia del meccanismo di **logit boosting**, 
è stato condotto un esperimento comparativo tra due varianti di generazione dello stesso modello:  
- **baseline** → decoding standard,  
- **boosted** → decoding con logit boosting guidato dalla rete di influenza ($\lambda=1.5$).  

Sono stati generati **100 campioni indipendenti per ciascuna variante** (200 in totale), 
usando lo stesso prompt di partenza (*"The future of AI is"*).  
Le coppie baseline/boosted sono state poi valutate da un **modello esterno in ruolo di giudice** 
(*Llama-3.1-8B-Instruct*), che assegna punteggi (scala 1–10) su tre dimensioni:  

- *coherence* → fluidità e consistenza logica del testo,  
- *informativeness* → quantità e rilevanza delle informazioni fornite,  
- *factuality* → correttezza e plausibilità delle affermazioni.  

Oltre ai punteggi del giudice, sono state calcolate **metriche automatiche aggiuntive**:  
- *token entropy* (diversità lessicale),  
- *prompt coverage* (riuso del lessico del prompt),  
- *semantic novelty* (distanza semantica tra baseline e boosted con SBERT).  

In questo modo il test combina una valutazione qualitativa (basata su un LLM 
usato come giudice) e quantitativa (metriche lessicali e semantiche), offrendo 
un quadro completo delle differenze introdotte dal boosting.

---

### Step 1: Generazione e salvataggio dei testi

Questa cella definisce due funzioni per gestire la generazione e il salvataggio dei testi:

- **`generate_variant`**: produce un output a partire da un prompt, scegliendo tra due varianti:
    -  *baseline* (decoding standard),
    -   *boosted* (decoding con logit boosting).


- **`save_generations`**: esegue più generazioni per ciascun prompt, in entrambe le varianti, e salva i risultati in formato **JSONL** con metadati (prompt_id, variante, indice_generazione, testo).

Serve quindi a creare in modo sistematico il dataset di generazioni necessario per i test comparativi.


In [ ]:
def generate_variant(model, tokenizer, prompt, variant, *,
                     G=None, score_bias=None,
                     max_new_tokens=200,    # baseline
                     lambda_=1.5, steps=200,      # boosted
                     seed=42):
    if seed is not None:
        import random, numpy as np, torch
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)

    if variant == "base":
        return baseline_decoding(
            model, tokenizer, prompt,
            max_new_tokens=max_new_tokens
        )
    elif variant == "boosted":
        return generate_with_logit_boosting(
            model, tokenizer, prompt, G, score_bias,
            lambda_=lambda_, steps=steps
        )
    else:
        raise ValueError("Variant must be 'base' or 'boosted'")


def save_generations(prompts, model, tokenizer, G, logit_bias, n=5, path="outputs.jsonl"):
    total = len(prompts) * n * 2   # numero totale di generazioni
    counter = 0

    with open(path, "w", encoding="utf-8") as f:
        for pid, prompt in enumerate(prompts):
            for i in range(n):
                for variant in ["base", "boosted"]:
                    counter += 1
                    print(f"[RUN] Generazione {counter}/{total} | prompt_id={pid}, sample={i}, variant={variant}")

                    text = generate_variant(
                        model, tokenizer, prompt, variant,
                        G=G, logit_bias=logit_bias, seed=i
                    )
                    rec = {
                        "prompt_id": pid,
                        "variant": variant,
                        "sample_id": i,
                        "prompt": prompt,
                        "text": text
                    }
                    f.write(json.dumps(rec, ensure_ascii=False) + "\n")

    print(f"[RUN] Completato! Generazioni salvate in: {path}")


Run delle generazioni:

In [ ]:
prompts = [
    "The future of AI is"
]

G = G
score_bias = score_bias

print("[RUN] Generazione in corso...")
save_generations(prompts, model, tokenizer, G, score_bias, n=50, path="outputs.jsonl")

print("[RUN] Generazioni salvate in outputs.jsonl")
with open("outputs_to_judge.jsonl", "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 5:  # preview
            break
        print(line.strip())

---

### Step 2: Setup del modello giudice e delle metriche aggiuntive

Questa cella si occupa del **setup del modello giudice e delle metriche aggiuntive**:

* Definisce l’ID del modello usato come *judge* (`Llama-3.1-8B-Instruct`).
* Configura la **quantizzazione 4-bit** tramite `BitsAndBytesConfig` per ridurre memoria e migliorare l’efficienza.
* Carica tokenizer e modello da HuggingFace, con mappatura automatica su GPU e valutazione in sola lettura (`.eval()`).
* Imposta il `pad_token_id` per garantire una gestione corretta delle sequenze.
* Infine, carica **SBERT** (`all-MiniLM-L6-v2`), che verrà utilizzato per calcolare la *semantic novelty*.

Si tratta dunque della cella di inizializzazione del modello giudice e dell’embedding model per le valutazioni.


In [ ]:
JUDGE_MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,   # usa 4-bit quantization
    bnb_4bit_compute_dtype="bfloat16",  # precisione dei calcoli
    bnb_4bit_use_double_quant=True,
)

print("[C1] Caricamento tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(JUDGE_MODEL_ID)

print("[C1] Caricamento modello...")
judge_model = AutoModelForCausalLM.from_pretrained(
    JUDGE_MODEL_ID,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)
judge_model.eval()

pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id

# === SBERT per novelty semantica ===
sbert = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

Questa cella definisce il **framework di valutazione automatica** delle coppie *baseline* vs *boosted* con il modello giudice:

* **`SYSTEM_PROMPT`**: istruzioni dettagliate fornite al modello giudice, che descrivono come confrontare le due uscite e restituire solo un JSON con i punteggi su *coherence*, *informativeness* e *factuality*.
* **`build_eval_prompt`**: costruisce il prompt di valutazione, includendo ID, testo di input e le due varianti da confrontare.
* **`parse_strict_json`**: funzione robusta che taglia eventuale testo extra e tenta di decodificare il JSON prodotto dal modello.
* **`judge_pair`**: funzione principale che

  1. combina le istruzioni di sistema con i testi da valutare,
  2. passa il prompt al modello giudice,
  3. acquisisce l’output generato,
  4. prova a fare il parsing in formato JSON,
  5. restituisce i punteggi strutturati o, in caso di errore, il testo grezzo.

Questa parte implementa il meccanismo che permette al modello giudice di valutare in modo standardizzato le coppie di generazioni e di restituire i risultati in un formato facilmente analizzabile.


In [ ]:
SYSTEM_PROMPT = """You are an evaluator. 
You MUST carefully compare the two outputs ("base" and "boosted") given the same prompt. 
For each output, assign integer scores (1–10) on coherence, informativeness, and factuality. 
Base your evaluation ONLY on the given prompt and the two outputs. Ignore any external knowledge.
Return ONLY a valid JSON object with this schema:
{
 "prompt_id": <int>,
 "sample_id": <int>,
 "base": {"coherence": <1-10>, "informativeness": <1-10>, "factuality": <1-10>},
 "boosted": {"coherence": <1-10>, "informativeness": <1-10>, "factuality": <1-10>}
}

Scoring rules:
- DO NOT give all 10s unless the text is truly flawless.
- Use the full 1–10 scale realistically.
- Different outputs should usually receive different scores.
- Penalize vagueness, repetition, or lack of relevance.
- Favor detailed, coherent, and factually plausible text.

Example of good scoring:
{
 "prompt_id": 99,
 "sample_id": 42,
 "base": {"coherence": 6, "informativeness": 5, "factuality": 7},
 "boosted": {"coherence": 8, "informativeness": 7, "factuality": 6}
}

No other text, no explanations, no markdown.
"""


def build_eval_prompt(prompt_id: int, sample_id: int, prompt: str, base_text: str, boosted_text: str) -> str:
    return (
        f"prompt_id: {prompt_id}\n"
        f"sample_id: {sample_id}\n\n"
        f"Prompt:\n{prompt}\n\n"
        f"Base output:\n{base_text}\n\n"
        f"Boosted output:\n{boosted_text}\n\n"
        "Now respond with ONLY the JSON object."
    )

def parse_strict_json(text: str):
    """
    Taglia il testo a [SYSTEM] (se presente) e prova a fare il parsing del JSON.
    """
    if "[SYSTEM]" in text:
        text = text.split("[SYSTEM]")[0]  # tieni solo la parte prima
    
    if "}" in text:
        candidate = text[:text.rfind("}")+1]  # fino all'ultima graffa chiusa
        try:
            return json.loads(candidate)
        except json.JSONDecodeError as e:
            print(f"[C1] JSONDecodeError: {e}")
            return None
    return None



def judge_pair(prompt_id: int, sample_id: int, prompt: str, base_text: str, boosted_text: str, max_new_tokens=128) -> Dict:
    user_prompt = build_eval_prompt(prompt_id, sample_id, prompt, base_text, boosted_text)
    full_prompt = f"[SYSTEM]\n{SYSTEM_PROMPT}\n\n[USER]\n{user_prompt}\n\n[ASSISTANT]\n"

    '''print("====== PROMPT PASSATO AL MODELLO ======")
    print(full_prompt)
    print("=======================================")'''

    inputs = tokenizer(full_prompt, return_tensors="pt").to(judge_model.device)
    input_len = inputs["input_ids"].shape[1]

    gen_out = judge_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        return_dict_in_generate=True,
    )

    new_tokens = gen_out.sequences[0][input_len:]
    raw_output = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    parsed = parse_strict_json(raw_output)

    print("====== OUTPUT PARSED DEL MODELLO ======")
    print(parsed)
    print("=======================================")

    if parsed and "base" in parsed and "boosted" in parsed:
        return parsed
    else:
        return {"error": raw_output}


Questa cella definisce funzioni di **pulizia del testo**:

* `_clean_text` rimuove tag `<s>`, `</s>` e spazi extra.
* `_tokens` tokenizza il testo in minuscolo, eliminando stopword e parole troppo corte.
  Serve per preparare le generazioni a calcoli come *prompt coverage* o altre metriche lessicali.


In [ ]:
# Utilità di pulizia

def _clean_text(t: str) -> str:
    if not isinstance(t, str):
        return ""
    t = t.replace("<s>", " ").replace("</s>", " ")
    return re.sub(r"\s+", " ", t).strip()

STOPWORDS = set("""
a an and are as at be by for from has have i in is it its of on or that the to was were will with this these those into about over under between within without against across among
""".split())

def _tokens(text: str):
    toks = re.findall(r"[a-zA-Z0-9']+", text.lower())
    return [t for t in toks if len(t) > 1 and t not in STOPWORDS]

Questa cella si occupa di definire le metriche aggiuntive calcolate:

- **Token entropy** (diversità lessicale)  
  Misura quanto è distribuita uniformemente la frequenza dei token:  
  $$
  H = - \sum_i p_i \log p_i
  $$ 
  con $p_i$ probabilità empirica del token $i$.


- **Semantic novelty** (distanza semantica tra baseline e boosted)  
  Basata su embedding SBERT e similarità coseno:  
  $$
  1 - \cos(\mathbf{e}_{\text{base}}, \mathbf{e}_{\text{boost}})
  $$
  Valori vicini a 0 indicano testi simili, valori alti segnalano maggiore novità.

In [ ]:
# Metriche extra
def token_entropy(text: str) -> float:
    toks = _tokens(text)
    if not toks: return 0.0
    _, counts = np.unique(toks, return_counts=True)
    probs = counts / counts.sum()
    return -(probs * np.log(probs)).sum()

def semantic_novelty(base_text: str, boosted_text: str) -> float:
    if not base_text.strip() or not boosted_text.strip():
        return 0.0
    emb = sbert.encode([base_text, boosted_text], convert_to_tensor=True, normalize_embeddings=True)
    sim = util.cos_sim(emb[0], emb[1]).item()
    return 1.0 - sim

### Step 3: Giudizio
Questa cella implementa il **processo di valutazione e arricchimento dei risultati**:

* Carica il file con le generazioni (`outputs_to_judge.jsonl`) e raggruppa le righe per coppia *(prompt\_id, sample\_id)*.
* Per ciascuna coppia **baseline/boosted**:

  * pulisce i testi,
  * li sottopone al modello giudice con `judge_pair`,
  * calcola le metriche extra (*token entropy, semantic novelty*).
* Crea due nuove righe arricchite (una per baseline e una per boosted) che includono sia i punteggi del judge sia le metriche automatiche.
* Salva tutto nel file di output `outputs_judged.jsonl`.
* Infine stampa un riepilogo con il numero di coppie valutate, errori di parsing e righe non accoppiate.

È la cella che produce i risutati finali per la fase di analisi comparativa


In [ ]:
# path
INPUT_JSONL  = "/kaggle/input/llm-as-a-judge/outputs_to_judge.jsonl"
OUTPUT_JSONL = "outputs_judged.jsonl"

# Caricamento input e accoppiamento base-boosted
rows = [json.loads(line) for line in open(INPUT_JSONL, "r", encoding="utf-8") if line.strip()]
print(f"[C2] Righe lette: {len(rows)} dal file: {INPUT_JSONL}")

buckets = defaultdict(list)
for r in rows:
    key = (r["prompt_id"], r["sample_id"])
    buckets[key].append(r)

augmented_rows = []
num_pairs, num_parse_errors, leftovers = 0, 0, 0

for (pid, sid), items in buckets.items():
    if len(items) < 2:
        leftovers += len(items)
        continue

    base_row   = next((r for r in items if r["variant"] == "base"), None)
    boosted_row= next((r for r in items if r["variant"] == "boosted"), None)
    if not base_row or not boosted_row:
        leftovers += len(items)
        continue

    base_text   = _clean_text(base_row["text"])
    boosted_text= _clean_text(boosted_row["text"])
    prompt      = base_row["prompt"]

    if num_pairs < 2:
        print(f"[C2] Pair (prompt_id={pid}, sample_id={sid})")
        print("Prompt:", (prompt[:120] + "...") if len(prompt) > 120 else prompt)
        print("Base:", (base_text[:80] + "...") if len(base_text) > 80 else base_text)
        print("Boosted:", (boosted_text[:80] + "...") if len(boosted_text) > 80 else boosted_text)

    try:
        judge = judge_pair(pid, sid, prompt, base_text, boosted_text)
    except Exception as e:
        print(f"[C2] ERRORE giudizio (pid={pid}, sid={sid}): {e}")
        num_parse_errors += 1
        continue

    # Metriche aggiornate
    ent_base, ent_boost = token_entropy(base_text), token_entropy(boosted_text)
    cov_base, cov_boost = prompt_coverage(base_text, prompt), prompt_coverage(boosted_text, prompt)
    sem_novelty = semantic_novelty(base_text, boosted_text)

    if not judge or "base" not in judge or "boosted" not in judge:
        print(f"[C2] Parsing fallito per (pid={pid}, sid={sid})")
        num_parse_errors += 1
        continue

    # riga base arricchita
    row_base = dict(base_row,
                    coherence=judge["base"]["coherence"],
                    informativeness=judge["base"]["informativeness"],
                    factuality=judge["base"]["factuality"],
                    entropy=ent_base,
                    prompt_coverage=cov_base,
                    semantic_novelty=sem_novelty)
    augmented_rows.append(row_base)

    # riga boosted arricchita
    row_boost = dict(boosted_row,
                     coherence=judge["boosted"]["coherence"],
                     informativeness=judge["boosted"]["informativeness"],
                     factuality=judge["boosted"]["factuality"],
                     entropy=ent_boost,
                     prompt_coverage=cov_boost,
                     semantic_novelty=sem_novelty)
    augmented_rows.append(row_boost)

    num_pairs += 1

# salvataggio
with open(OUTPUT_JSONL, "w", encoding="utf-8") as fout:
    for r in augmented_rows:
        fout.write(json.dumps(r, ensure_ascii=False) + "\n")

print("-" * 70)
print(f"[C2] Coppie giudicate: {num_pairs} | Errori parsing: {num_parse_errors} | Non accoppiate: {leftovers}")
print(f"[C2] File con giudizi salvato in: {OUTPUT_JSONL}")


### Step 4: Analisi Comparativa

Questa cella carica i risultati giudicati, li raggruppa per coppia *baseline/boosted* e calcola le medie delle metriche principali. I valori vengono salvati in CSV e visualizzati tramite grafici a barre (sia multipli che singoli) per confrontare rapidamente le due varianti su coerenza, informatività, factualità ed extra metriche (*entropy, prompt coverage, semantic novelty*).


In [ ]:
INPUT = Path("/kaggle/input/llm-as-a-judge/outputs_judged.jsonl")
OUTDIR = Path("./figures")
OUTDIR.mkdir(exist_ok=True)

# caricamento json
rows = [json.loads(l) for l in INPUT.read_text(encoding="utf-8").splitlines() if l.strip()]
print(f"[C3] Righe caricate: {len(rows)} da {INPUT}")

def is_number(x):
    try:
        float(x); return True
    except:
        return False

# raggruppamento (coppie)
groups = defaultdict(list)
for r in rows:
    key = r.get("pair_id") or r.get("prompt") or r.get("prompt_text") or r.get("input") or "NO_KEY"
    groups[key].append(r)

classic_metrics = ["coherence", "informativeness", "factuality"]
candidate_extra = ["entropy", "semantic_novelty"]
present_extra = [m for m in candidate_extra if any(m in r for r in rows)]

# variant-wise
var_values = defaultdict(lambda: defaultdict(list))

for items in groups.values():
    by_var = defaultdict(list)
    for it in items:
        if "variant" in it:
            by_var[it["variant"].lower()].append(it)

    if "base" in by_var and "boosted" in by_var:
        for b, s in zip(by_var["base"], by_var["boosted"]):
            
            # metriche classiche
            for m in classic_metrics:
                if m in b and m in s and is_number(b[m]) and is_number(s[m]):
                    var_values["base"][m].append(float(b[m]))
                    var_values["boosted"][m].append(float(s[m]))

            # entropy 
            for m in ["entropy"]:
                if m in present_extra:
                    if is_number(b.get(m)) and is_number(s.get(m)):
                        var_values["base"][m].append(float(b[m]))
                        var_values["boosted"][m].append(float(s[m]))

# semantic_novelty
novelty_vals = []
for r in rows:
    if "semantic_novelty" in r and is_number(r["semantic_novelty"]):
        novelty_vals.append(float(r["semantic_novelty"]))

# tabella media semplice 
records = []

for m in classic_metrics + ["entropy", "prompt_coverage"]:
    if m in present_extra or m in classic_metrics:
        for var in ["base", "boosted"]:
            vals = var_values[var].get(m, [])
            if vals:
                mean = np.mean(vals)
                records.append({"metric": m, "variant": var, "mean": round(mean, 3)})

if novelty_vals:
    mean = np.mean(novelty_vals)
    records.append({"metric": "semantic_novelty", "variant": "boosted_only", "mean": round(mean, 3)})

df = pd.DataFrame(records)
print("\n=== Valori medi")
print(df)

df.to_csv(OUTDIR / "metrics_summary_simple.csv", index=False)

# plot
metrics_to_plot = classic_metrics + [m for m in ["entropy"] if m in present_extra]
colors = {"base": "#007aff", "boosted": "#ff4d4d"}

ncols = 2
nrows = int(np.ceil(len(metrics_to_plot) / ncols))

fig, axes = plt.subplots(nrows, ncols, figsize=(6*ncols, 5*nrows), sharey=False)
axes = axes.flatten()

for ax, m in zip(axes, metrics_to_plot):
        means, xs = [], []
        for i, var in enumerate(["base", "boosted"]):
            vals = var_values[var].get(m, [])
            if vals:
                xs.append(i)
                means.append(np.mean(vals))

        # plot nel grafico multiplo
        ax.bar(xs, means, color=[colors[v] for v in ["base", "boosted"]])
        ax.set_xticks(xs)
        ax.set_xticklabels(["Base", "Boosted"])
        ax.set_title(m)
        if m in ["coherence", "informativeness", "factuality", "entropy"]:
            ax.set_ylim(0, 10)
        ax.grid(axis="y", linestyle="--", alpha=0.6)

        for bar, val in zip(ax.patches, means):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                    f"{val:.2f}", ha="center", va="bottom", fontsize=10, fontweight="bold")

        # plot e salvataggio singolo
        fig_single, ax_single = plt.subplots(figsize=(5,5))
        bars_single = ax_single.bar(xs, means, color=[colors[v] for v in ["base", "boosted"]])
        ax_single.set_xticks(xs)
        ax_single.set_xticklabels(["Base", "Boosted"])
        ax_single.set_title(m)
        ax_single.grid(axis="y", linestyle="--", alpha=0.6)

        if m in ["coherence", "informativeness", "factuality", "entropy"]:
            ax_single.set_ylim(1, 10)

        for bar, val in zip(bars_single, means):
            ax_single.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                           f"{val:.2f}", ha="center", va="bottom", fontsize=10, fontweight="bold")

        fig_single.savefig(OUTDIR / f"{m}.png", dpi=150, bbox_inches="tight")
        plt.close(fig_single)

---

# 7. Ottimizzazioni

## 7.1 Logits Processor

La libreria `transformers` permette di personalizzare la fase di generazione
tramite i **LogitsProcessor**, moduli che modificano i punteggi del modello
(logits) prima della scelta del token successivo. Sono lo strumento pensato per
applicare vincoli o bias senza dover riscrivere manualmente il ciclo di decoding.  

In questo lavoro è stata implementata una versione personalizzata,
**GraphBiasLogitsProcessor**, che integra l’informazione della rete di influenza
direttamente nel processo di generazione. In questo modo il boosting viene
applicato in maniera trasparente e nativa, con la possibilità di combinarlo
facilmente con altre tecniche di campionamento.  


Questa cella definisce l’integrazione del boosting tramite `LogitsProcessor`:

* **`GraphBiasLogitsProcessor`**: sottoclasse di `LogitsProcessor` che modifica i logits del modello aggiungendo un bias proporzionale ai pesi della rete di influenza (`G`) e a un termine esterno (`logit_bias`), scalati da `λ`.
* **`generate_with_boosting_fast`**: funzione di generazione che inserisce il processore personalizzato dentro `transformers.generate`, così il boosting viene applicato automaticamente durante la scelta dei token, combinabile con strategie di campionamento (`top-k`, ecc.).

In sintesi, è la versione “veloce e nativa” del logit boosting.


In [ ]:
class GraphBiasLogitsProcessor(LogitsProcessor):
    def __init__(self, G, score_bias, lambda_, tokenizer=None):
        self.G = G
        self.score_bias = score_bias
        self.lambda_ = lambda_
        self.tokenizer = tokenizer  # solo per le stampe leggibili

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        current_token = input_ids[0, -1].item()  # ultimo token generato
        boosted_scores = scores.clone()

        # seleziona solo i top-100 candidati
        topk = torch.topk(scores, 100)
        indices = topk.indices[0].tolist()
        values = topk.values[0].tolist()

        print("\n=== Nuovo step ===")
        print("--- Top 20 PRIMA boosting ---")
        for idx, val in zip(topk.indices[0][:20].tolist(), topk.values[0][:20].tolist()):
            token_str = self.tokenizer.decode([idx]) if self.tokenizer else str(idx)
            print(f"{idx}:'{token_str}' → logit={val:.3f}")

        # applica boosting solo sui top-100
        for idx, val in zip(indices, values):
            infl = self.score_bias.get(idx, 0.0)
            w = 0.0
            if current_token in self.G and idx in self.G[current_token]:
                w = self.G[current_token][idx].get("weight", 0.0)
            boost = w + infl
            if boost > 0:
                boosted_scores[0, idx] = val + self.lambda_ * boost

        top20_after = torch.topk(boosted_scores, 20)
        print("\n--- Top 20 DOPO boosting ---")
        for idx, val in zip(top20_after.indices[0].tolist(), top20_after.values[0].tolist()):
            token_str = self.tokenizer.decode([idx]) if self.tokenizer else str(idx)
            print(f"{idx}:'{token_str}' → logit={val:.3f}")

        return boosted_scores


def generate_with_boosting_fast(model, tokenizer, prompt, G, score_bias, lambda_=2.0, max_new_tokens=50):
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    processor = GraphBiasLogitsProcessor(G, score_bias, lambda_, tokenizer)

    output_ids = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True, 
        top_k=50,
        logits_processor=LogitsProcessorList([processor])
    )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)


In [ ]:
generate_with_boosting_fast(model, tokenizer, prompt, G, score_bias, lambda_=1.5, max_new_tokens= 200) 

Per verificarne i benefici, è stato condotto un confronto con l’approccio manuale: a parità di condizioni ($\lambda=1.5$, sequenza di ~200 token), il metodo tradizionale ha richiesto oltre 3 minuti, mentre con LogitsProcessor il tempo è stato ridotto a meno di 20 secondi, con un guadagno di efficienza pari a circa **88.9%**.

| **Metodo**                        | **Tempo (mm:ss)** | **Speedup**   |
|-----------------------------------|-------------------|---------------|
| Manuale (ciclo python esplicito)  | 03:07.30          | 1.0×          |
| Con LogitsProcessor (C++/Cuda)    | 00:18.97          | ~9.9×         |

Il miglioramento è spiegabile dal fatto che la funzione `model.generate()` sfrutta codice ottimizzato in C++ e CUDA, molto più efficiente rispetto a un ciclo Python puro, consentendo uno speedup rilevante nei tempi di generazione.
